In [8]:
search_for = 88
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004262685775756836
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 175
all cases: 190216155
type: [1, 1, 1, 1, 88] 88
cases of this type: 59969536
10000 30.054144386143122
20000 30.81133885315468
30000 103.26992892822652
40000 117.34169976343154
50000 116.47153805211249
60000 121.71779538528233
70000 113.05077064541832
80000 117.72637457217435
90000 111.2569577216978
100000 115.76641512330687
110000 109.24551350835587
120000 109.23030463367344
130000 108.85564942376345
140000 109.6333976039663
150000 111.11881536496391
160000 113.37267393600587
170000 117.91532248935452
180000 128.07256285760047
190000 112.8920805577448
200000 109.23553530355716
210000 114.9559748319
220000 111.76717996924087
230000 104.55247422097311
240000 107.10905514196972
250000 109.69320800345561
260000 105.96703916662163
270000 115.22872057357966
280000 108.3370543656709
290000 111.6150975171125
300000 105.93833194755167
310000 103.88

3080000 16.618503546370963
3090000 17.719653200692534
3100000 114.0543236406167
3110000 87.7340051482794
3120000 97.01056638239174
3130000 92.33914149759511
3140000 105.02380810839469
3150000 93.34479840212524
3160000 92.63406408200261
3170000 92.80717958415336
3180000 87.28127318549215
3190000 90.07225945620102
3200000 92.68297051372657
3210000 90.74034058933645
3220000 95.50699368480656
3230000 100.68782742443787
3240000 97.82649923498064
3250000 99.27975729435099
3260000 101.09248711449906
3270000 98.75151674167334
3280000 92.73612490360233
3290000 93.40457013029989
3300000 90.42769380316872
3310000 92.34988948992873
3320000 92.67570265883154
3330000 97.29404641139865
3340000 95.55157278206994
3350000 93.3744230763378
3360000 113.7321115660727
3370000 42.61198446818743
3380000 40.32821573913068
3390000 38.35671002061178
3400000 39.71927464765314
3410000 132.66868311640363
3420000 114.95045377626738
3430000 118.25804928425478
3440000 32.80448071224087
3450000 31.792968398666382
34600

6230000 64.62437163319608
6240000 102.7082615774095
6250000 92.19197867289262
6260000 89.77756049534126
6270000 93.64831173521291
6280000 99.68875159082336
6290000 94.625176163598
6300000 87.77057568256318
6310000 91.69198291363192
6320000 92.11786196097003
6330000 89.44007783041685
6340000 96.1173449453232
6350000 98.85325929139779
6360000 94.42236822725832
6370000 84.66408905974501
6380000 90.97232082146866
6390000 85.38087757902204
6400000 86.03141094209886
6410000 86.65673212072521
6420000 89.00758655936927
6430000 83.76804917930116
6440000 75.92881250511088
6450000 87.6095267089363
6460000 88.49036823941917
6470000 15.871304999169972
6480000 15.81396298857808
6490000 15.933364220593736
6500000 44.60877618666099
6510000 95.38518869993509
6520000 85.33843056623238
6530000 79.14206717349195
6540000 83.5535955902631
6550000 86.9319280063201
6560000 85.92426001498013
6570000 84.39951567736784
6580000 81.79993253656976
6590000 89.61397121417082
6600000 90.43052595600605
6610000 90.47641

9380000 94.78272460337793
9390000 93.09571854338854
9400000 95.821060995274
9410000 99.48383545580838
9420000 102.40816236893257
9430000 67.46768916827408
9440000 38.67338492907749
9450000 38.16673075193683
9460000 39.32212237978439
9470000 90.80926130481657
9480000 95.43371898352743
9490000 95.4274560487019
9500000 92.43623925102459
9510000 90.60884271624137
9520000 93.45536082412958
9530000 92.88387675515165
9540000 101.86730369403342
9550000 97.80017715430336
9560000 99.65464534404504
9570000 94.3826722903949
9580000 87.29311493816196
9590000 96.29210032094993
9600000 91.16207721769364
9610000 92.10190207453903
9620000 91.47931038952606
9630000 99.56614980003039
9640000 59.78590196501519
9650000 31.821531075102683
9660000 31.90821194775671
9670000 31.579420453187257
9680000 103.59984562234305
9690000 93.84456614430964
9700000 86.13168074002554
9710000 92.47907743168341
9720000 97.06455775529781
9730000 88.39543265301751
9740000 86.47148693431262
9750000 92.09243474704473
9760000 90.

12450000 90.27324713895172
12460000 87.35629848159375
12470000 89.7786503224298
12480000 83.31372966534664
12490000 83.18298655711463
12500000 88.92587201002118
12510000 84.14729978640129
12520000 93.43881720690435
12530000 81.49395916420404
12540000 78.70600378035664
12550000 83.94471194915496
12560000 81.30442390573515
12570000 83.0806214969626
12580000 75.48578002477332
12590000 98.77616452037887
12600000 30.507875907324355
12610000 15.424304262704618
12620000 15.529488961564732
12630000 15.497669992883207
12640000 98.6095684258866
12650000 88.33530063665346
12660000 76.51473810153624
12670000 87.18082083858484
12680000 78.2077064484805
12690000 85.19350894449136
12700000 78.944741686391
12710000 83.95137522972233
12720000 91.28497980844975
12730000 82.10603672296875
12740000 86.46158397996008
12750000 82.341905316784
12760000 92.37199488199188
12770000 87.59611213883655
12780000 85.46234288815428
12790000 99.60487832053136
12800000 58.615161003664475
12810000 37.97598954581559
1282

...

179440000 1.0824895254194404
179450000 0.7250630718901422
best so far: 0
type: [1, 4, 11, 1, 2] 88
cases of this type: 7744
179460000 0.7177913750773668
best so far: 0
type: [1, 4, 11, 2, 1] 88
cases of this type: 3872
best so far: 0
type: [1, 4, 22, 1, 1] 88
cases of this type: 1936
best so far: 0
type: [1, 8, 1, 1, 11] 88
cases of this type: 117128
179470000 0.8038261430483726
179480000 1.3422032466985783
179490000 1.3736565832704306
179500000 1.3792987194698716
179510000 0.9129665274402167
179520000 0.8158264586315552
179530000 0.8292893185408579
179540000 0.8334896637604303
179550000 0.8353912009503444
179560000 1.163416843994922
179570000 1.2124134464833471
179580000 1.3037392191579937
best so far: 0
type: [1, 8, 1, 11, 1] 88
cases of this type: 10648
179590000 0.9932565876504447
best so far: 0
type: [1, 8, 11, 1, 1] 88
cases of this type: 968
best so far: 0
type: [1, 11, 1, 1, 8] 88
cases of this type: 45056
179600000 0.8731089227653543
179610000 1.160790869560341
179620000 0.760

181680000 0.6506714333604773
181690000 0.6399358983708753
181700000 0.6194119728714228
181710000 0.62812599480172
181720000 0.5476310301228033
181730000 0.6390537656887704
181740000 0.6336894722825289
181750000 0.6473114786460002
181760000 0.6805579575274057
181770000 0.5955628612644474
181780000 0.6478004001553852
181790000 0.6644117340021664
181800000 0.6652481525163849
181810000 0.6292424801582097
181820000 0.6569198147908847
181830000 0.590591359640956
181840000 0.6477129763886995
181850000 0.6361384050144752
181860000 2.7508111087855696
181870000 2.5913604572930935
181880000 2.5725051517653794
181890000 1.8064524564379456
181900000 0.7331578831118677
181910000 2.000422722526954
181920000 2.516388338267505
181930000 2.357856832494471
181940000 2.5546984715445835
181950000 2.234726585768461
181960000 2.199695308843189
181970000 2.1761810177000034
181980000 1.0521642141580583
181990000 1.1179157839871114
182000000 2.1437298452629645
182010000 2.14885211604774
182020000 2.289740362328

184510000 0.44997664010812843
184520000 0.4402440039272441
184530000 0.42505458213776354
184540000 0.4553909477627609
184550000 0.39911334392199915
184560000 0.4332578877616922
184570000 0.43552607486844064
184580000 0.4463649781619178
184590000 0.42203112338205173
184600000 0.4075206990109218
184610000 0.40024233884311383
184620000 0.4123915424492955
184630000 0.4100442396747404
184640000 0.42894446529322205
184650000 0.4511405412832895
184660000 0.44435170608533753
184670000 1.1046222553307812
184680000 1.0767529769258697
184690000 1.069810205314623
184700000 0.7501965179909269
184710000 0.466788761651814
184720000 0.7542487013137341
184730000 1.0841634204805228
184740000 1.1460629602470995
184750000 1.1520359531380733
184760000 1.0540044386629264
184770000 0.9702811958017946
184780000 0.8371651922979952
184790000 0.5600221675268147
184800000 0.4632860075187683
184810000 0.972670097057091
184820000 0.9431202327365344
184830000 1.0189662685272098
184840000 1.0862877427272997
184850000

187210000 0.4671954230358203
187220000 0.6003866656002071
187230000 0.6258540800467134
187240000 0.3912626549831364
187250000 0.3933465581241912
187260000 0.5417915519973636
187270000 0.2189008708107471
187280000 0.22378534085528717
187290000 0.22673910376469295
187300000 0.2033048923164275
187310000 0.2106161434959041
187320000 0.21601284904450177
187330000 0.2175508925467067
187340000 0.18796359433995352
187350000 0.22815705103586115
187360000 0.22960129637151958
187370000 0.20234304412884843
187380000 0.21294343114634354
187390000 0.22425098568873272
187400000 0.21604729663024344
187410000 0.21746171454042196
187420000 0.21591359705140195
187430000 0.2098329863271448
187440000 0.22030744685967765
187450000 0.21277258594724868
187460000 0.20776080969638291
187470000 0.19704498308092355
187480000 0.47637433903409376
187490000 0.5530904647196333
187500000 0.27222006439477203
187510000 0.5495358082553082
187520000 0.5666251413065526
187530000 0.4829206989520788
187540000 0.3200997690419

best so far: 0
type: [2, 2, 2, 11, 1] 88
cases of this type: 10648
189620000 0.058185819535553455
best so far: 0
type: [2, 2, 11, 1, 2] 88
cases of this type: 3872
189630000 0.03708323531220356
best so far: 0
type: [2, 2, 11, 2, 1] 88
cases of this type: 1936
best so far: 0
type: [2, 2, 22, 1, 1] 88
cases of this type: 968
best so far: 0
type: [2, 4, 1, 1, 11] 88
cases of this type: 58564
189640000 0.05995342296567228
189650000 0.07046312738196718
189660000 0.04348521185338497
189670000 0.0415860790263613
189680000 0.045980733057492305
189690000 0.06907426172951857
best so far: 0
type: [2, 4, 1, 11, 1] 88
cases of this type: 5324
best so far: 0
type: [2, 4, 11, 1, 1] 88
cases of this type: 484
best so far: 0
type: [2, 11, 1, 1, 4] 88
cases of this type: 2816
189700000 0.0419415652000573
best so far: 0
type: [2, 11, 1, 2, 2] 88
cases of this type: 1408
best so far: 0
type: [2, 11, 1, 4, 1] 88
cases of this type: 704
best so far: 0
type: [2, 11, 2, 1, 2] 88
cases of this type: 704
best s